## Cryptocurrency Price Visualization Tool ###

In [2]:
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import plot
import requests
from datetime import datetime

crypto_id = None

def available_crypto_ids():
    url = "https://min-api.cryptocompare.com/data/all/coinlist"
    response = requests.get(url)
    data = response.json()["Data"]
    return list(data.keys())

def available_fiat_currencies():
    url = "https://openexchangerates.org/api/currencies.json"
    response = requests.get(url)
    data = response.json()
    fiat_currencies = list(data.keys())
    return fiat_currencies

def fetch_and_process_data(currency, start_date, end_date, aggregation):
    global crypto_id
    
    available_ids = available_crypto_ids()
    if crypto_id not in available_ids:
        raise ValueError(f"Invalid cryptocurrency ID. Available IDs: {', '.join(available_ids)}")
        
    valid_currencies = available_fiat_currencies()
    if currency.upper() not in valid_currencies:
        raise ValueError(f"Invalid currency. Available currencies: {', '.join(valid_currencies)}")
        
    url = f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={crypto_id}&tsym={currency}&limit=2000"
    response = requests.get(url)
    data = response.json()["Data"]["Data"]
    df = pd.DataFrame(data)
    
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df.set_index("time", inplace=True)

    filtered_data = df[start_date:end_date]

    if aggregation == "daily":
        aggregation_data = filtered_data.resample("D").agg({
            "open": "first",
            "high": "max",
            "low": "min",
            "close": "last"
        }).dropna()
        return aggregation_data
    
    elif aggregation == "monthly":
        aggregation_data = filtered_data.resample("M").agg({
            "open": "first",
            "high": "max",
            "low": "min",
            "close": "last"
        }).dropna()
        return aggregation_data
    
    elif aggregation == "yearly":
        aggregation_data = filtered_data.resample("Y").agg({
            "open": "first",
            "high": "max",
            "low": "min",
            "close": "last"
        }).dropna()
        return aggregation_data


def candlestick_chart(data):
    fig = go.Figure(data=[go.Candlestick(x=data.index,
                                         open=data['open'],
                                         high=data['high'],
                                         low=data['low'],
                                         close=data['close'])
                          ])

    fig.update_layout(
        title= f"Cryptocurrency Price Candlestick Chart of {crypto_id}",
        xaxis_title="Date",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False
    )

    fig.show()


def main():
    try:
        global crypto_id
        print("Welcome to Cryptocurrency Price Visualization Tool!\n")
        crypto_id = input("Enter Cryptocurrency ticker symbol (e.g., BTC, ETH): ").upper()
        currency = input("Enter Currency (e.g., USD, EUR, INR): ").upper()
        start_date = datetime.strptime(input("Enter Start Date (YYYY-MM-DD): "), "%Y-%m-%d")
        end_date = datetime.strptime(input("Enter End Date (YYYY-MM-DD): "), "%Y-%m-%d")
        aggregation = input("Enter Aggregation (daily, monthly, yearly): ").lower()  

        data = fetch_and_process_data(currency, start_date, end_date, aggregation)

        if data is not None and not data.empty:
            candlestick_chart(data)
        else:
            print("No data available for the specified date range.")
            
    except ValueError as ve:
        print("Error:", ve)
    except requests.RequestException as re:
        print("API Error:", re)
    except Exception as e:
        print("An error occurred:", e)

    print("\nThank you for using Cryptocurrency Price Visualization Tool!")


main()


